# OpenARK SQL Query Example

In this notebook, you can study some `OpenARK` features as below:

1. Load your Global Namespace
    - Parse the online account information from environment variables
    - Parse the kubernetes account information from files
1. Query with all your datasets in Global Namespace
    - SQL Engine: integrated in `polars`
    - Use `%%sql` iPython magic to simply query and visualize the data.
1. Use `polars` DataFrames as `pandas`
    - `LazyFrame` vs `DataFrame`

## Requirements

* `OpenARK` python client library
    ```bash
        $ pip install openark
    ```
* `OpenARK` online session: one of
    - `VINE` desktop
    - `VINE` remote SSH session
* `optimizer-metric` model (example dataset)

## Import Libraries

The libraries below are going to be imported:

* openark: to load and query your own global datasets


In [ ]:
from openark import OpenArk

## Initialize OpenARK Client

The `OpenArk` class simply retrieves user information in a simple way and provides all the convenient options available as methods.
When you instantiate this class, it automatically collects information about the `VINE` session you belong to.
Specifically:

1. Parse the online account information from environment variables
1. Parse the kubernetes account information from files
1. If the Python executor is running in `iPython`, various convenient magic items such as `%%sql` are added to iPython.

In [ ]:
# Create a OpenARK instance
openark = OpenArk()

## Query with `%%sql` magic

`SQL` is the easiest way to intuitively view and analyze your data. 
You can leverage any dataset you have access to using SQL queries.
SQL commands can be used through the `sql` method on the `openark` object,
but you can also execute and visualize SQL statements directly in a paragraph of code by inserting the `%%sql` keyword at the top of the code,
as in the example below.

In [ ]:
%%sql

SELECT name, namespace, kind, type, op, sum((end_ns - begin_ns) / 1000000000.0) as elapsed_sec, sum(len) as bytes, count() as len
FROM optimizer_metric
GROUP BY name, namespace, kind, type, op
ORDER BY name, namespace, kind, type, op

### Query and Visualize with other styles

`%%sql` magic can interpret, execute and visualize SQL statements.
But you may want to visualize the results in another forms such as plots .
It is possible by adding the desired style after the `%%sql` keyword, as shown in the example below.

In [ ]:
%%sql timeseries(y='mbps', hue='op')

SELECT __timestamp, name, namespace, kind, type, op, len * 1000.0 / (end_ns - begin_ns) as mbps
FROM optimizer_metric
WHERE kind = 'Storage' AND op = 'Put'
ORDER BY name, namespace, kind, type, op

## Query with `OpenArk` instance

`%%sql` magic is definitely useful for easily visualizing and analyzing data.
However, you may want to perform more advanced processing of the processed data using other modules and algorithms in Python.
You can use the `get_global_namespace` method of an `OpenArk` instance to obtain an `OpenArkGlobalNamespace` instance containing all available dataset metadata.

The `OpenArkGlobalNamespace` class supports various high-level methods for SQL queries.
Among them, you can obtain lazy (*unprocessed original*) frames by using the `sql` method.

The `LazyFrame` has a plan to analyze the schema of the dataset and process data that matches the query.
In addition, various optimization processes are applied to the LazyFrame, such as unifying redundant data processing prior to execution.
In this way, the LazyFrame can be seen as the state just before processing and executing SQL.

In [ ]:
# get your own global namespace
gn = openark.get_global_namespace()

# preprocess the SQL
my_dataset = 'optimizer_metric'
my_group = ','.join(['name', 'namespace', 'kind', 'type', 'op'])
unit_ns = 1000_000_000.0

# parse the SQL into `LazyFrame`
lf = gn.sql(f'''
SELECT {my_group}, sum((end_ns - begin_ns) / {unit_ns}) as elapsed_sec, sum(len) as bytes, count() as len
FROM {my_dataset}
GROUP BY {my_group}
ORDER BY {my_group}
''')
lf

### Execute the LazyFrame

To *locally* execute a query contained in a LazyFrame object,
you can perform the `collect` method as shown in the example below.

In [ ]:
df = lf.collect()
df

### Execute and Visualize

As in the case of the `%%sql` magic keyword, the `sql_and_draw` method allows you to collect the results and even visualize them in your desired style.
The example below guides you through how to apply the same style as in the *Magic* case above.

In [ ]:
sql = f'''
SELECT __timestamp, {my_group}, len * {1_000_000.0 * unit_ns} / (end_ns - begin_ns) as mbps
FROM optimizer_metric
WHERE kind = 'Storage' AND op = 'Put'
ORDER BY {my_group}
'''

gn.sql_and_draw(sql, style="timeseries(y='mbps', hue='op')")